In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.optim.lr_scheduler as lr_scheduler
import math
import random

MIN_LIST_LEN = 16
MAX_LIST_LEN = 16
MAX_STEPS = 640

SUCCESS_REWARD = 0.5
STEP_REWARD = -0.3
COMPARISON_ENTROPY_MULTIPLIER = -0.00
SWAP_REWARD = 1.0
INVALID_ACTION_REWARD = -10.0
LONGTERM_GAMMA = 0.99
SHORTTERM_GAMMA = 0.7

EPS_START = 0.5
EPS_END = 0.05
EPS_DECAY = 1000
LR_SCHEDULER_GAMMA = 0.96
NUM_EPISODES = 200000
EPISODES_SAVE = 1000
OUTPUT_DIR = '/home/mcwave/code/autocode/datasets/rl_sort_transformer_easy/list16_transformer4_192_gamma07_step640_v3'

# Define the vocabulary
vocab = {
    'Comparison': 0,
    'Swap': 1,
    'less': 2,
    'equal': 3,
    'more': 4,
    '0': 5,
    '1': 6,
    '2': 7,
    '3': 8,
    '4': 9,
    '5': 10,
    '6': 11,
    '7': 12,
    '8': 13,
    '9': 14,
    '10': 15,
    '11': 16,
    '12': 17,
    '13': 18,
    '14': 19,
    '15': 20,
    'len1': 21,
    'len2': 22,
    'len3': 23,
    'len4': 24,
    'len5': 25,
    'len6': 26,
    'len7': 27,
    'len8': 28,
    'len9': 29,
    'len10': 30,
    'len11': 31,
    'len12': 32,
    'len13': 33,
    'len14': 34,
    'len15': 35,
    'len16': 36,
}
inv_vocab = {v: k for k, v in vocab.items()}

def compute_entropy(N, alpha=1):
    K = 2**N
    values = np.arange(K)
    unnormalized_probs = np.exp(-alpha * values)
    Z = unnormalized_probs.sum()
    probs = unnormalized_probs / Z
    return values, -np.log2(probs)

_, int_entropy = compute_entropy(4)

def get_entropy_of_integer(x):
    x = min(15, abs(x))
    return int_entropy[x]

def compute_min_delta_entropy(comparisons):
    # Initialize the result list to store minDelta values
    min_delta = None

    # Iterate through each pair in the comparisons list
    i = len(comparisons) - 1
    xi, yi = comparisons[i]
    if i == 0:
        # For i = 0, use the first case directly
        min_delta = (xi, min(yi, yi - xi), 0)
    else:
        # For i > 0, compute all possible options and select the minimal one
        options = []

        # Simple Entropy
        simple_entropy = (xi, min(yi, yi - xi), 0)
        options.append(simple_entropy)

        # First Delta Entropy
        xi_prev, yi_prev = comparisons[i - 1]
        first_delta_entropy = (xi - xi_prev, yi - yi_prev, 0)
        options.append(first_delta_entropy)

        # Second Delta Entropy (only valid for i > 1)
        if i > 1:
            xi_prev2, yi_prev2 = comparisons[i - 2]
            second_delta_entropy = (
                (xi - xi_prev) - (xi_prev - xi_prev2),
                (yi - yi_prev) - (yi_prev - yi_prev2),
                0,
            )
            options.append(second_delta_entropy)

        # Arbitrary Position Entropy (only valid for i > 1)
        for j in range(i):
            xj, yj = comparisons[j]
            arbitrary_position_entropy = (
                xi - xj,
                yi - yj,
                min(j, i - j),
            )
            options.append(arbitrary_position_entropy)

        # Find the option with the minimal sum
        min_delta = min(options, key=lambda t: sum([get_entropy_of_integer(x) for x in t]))

    entropy = sum([get_entropy_of_integer(x) for x in min_delta])
    if len(comparisons) == 1:
        return 3 * entropy
    else:
        return entropy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define the environment
class SortingEnv:
    def __init__(self):
        self.max_steps = MAX_STEPS

    def reset(self):
        self.length = random.randint(MIN_LIST_LEN, MAX_LIST_LEN)
        self.list = [random.randint(1, 100) for _ in range(self.length)]
        while self.list == sorted(self.list):
            self.list = [random.randint(1, 100) for _ in range(self.length)]
        self.indices = None
        self.current_step = 0
        self.done = False
        initial_token = 'len{}'.format(self.length)
        return vocab[initial_token], self.list.copy()
    
    def get_list(self):
        return self.list
    
    def get_list_len(self):
        return len(self.list)

    def step(self, action_tokens):
        action = action_tokens[0]
        reward = -0.01  # default penalty
        response_token = None

        if action == vocab['Comparison']:
            if len(action_tokens) != 3:
                reward = INVALID_ACTION_REWARD
                self.done = True
                return response_token, reward, self.done, self.list.copy()
            index1 = action_tokens[1] - vocab['0']
            index2 = action_tokens[2] - vocab['0']
            if index1 >= self.length or index2 >= self.length or index1 < 0 or index2 < 0:
                reward = INVALID_ACTION_REWARD
                self.done = True
                return response_token, reward, self.done, self.list.copy()
            self.indices = (index1, index2)
            if self.list[index1] < self.list[index2]:
                response_token = vocab['less']
                reward = STEP_REWARD
            elif self.list[index1] == self.list[index2]:
                response_token = vocab['equal']
                reward = STEP_REWARD * 2
            else:
                response_token = vocab['more']
                reward = STEP_REWARD
        elif action == vocab['Swap']:
            if self.indices is None:
                reward = INVALID_ACTION_REWARD
                self.done = True
                return response_token, reward, self.done, self.list.copy()
            index1, index2 = self.indices
            prev_list = self.list.copy()
            self.list[index1], self.list[index2] = self.list[index2], self.list[index1]
            if self.list == sorted(self.list):
                reward = SUCCESS_REWARD
                self.done = True
            #elif prev_list[index1] > prev_list[index2] and self.list[index1] <= self.list[index2]:
            #    reward = 0.1
            elif (index1 < index2 and prev_list[index1] > prev_list[index2] and self.list[index1] <= self.list[index2]) or \
                (index1 > index2 and prev_list[index1] < prev_list[index2] and self.list[index1] >= self.list[index2]):
                reward = SWAP_REWARD
            elif (index1 < index2 and prev_list[index1] < prev_list[index2] and self.list[index1] >= self.list[index2]) or \
                (index1 > index2 and prev_list[index1] > prev_list[index2] and self.list[index1] <= self.list[index2]):
                reward = -SWAP_REWARD
            else:
                reward = STEP_REWARD
            self.indices = None
        else:
            reward = INVALID_ACTION_REWARD
            self.done = True

        self.current_step += 1
        if self.current_step >= self.max_steps:
            self.done = True
        return response_token, reward, self.done, self.list.copy()


Using device: cuda


In [2]:
# Positional Encoding for Transformer
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=640):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=0.1)

        pe = torch.zeros(max_len, d_model)  # (max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)  # (max_len, 1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() *
                             (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)  # Even indices
        pe[:, 1::2] = torch.cos(position * div_term)  # Odd indices
        pe = pe.unsqueeze(1)  # (max_len, 1, d_model)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

# Transformer Model
class TransformerModel(nn.Module):
    def __init__(self, vocab_size, d_model=192, nhead=8, num_layers=4):
        super(TransformerModel, self).__init__()
        self.model_type = 'Transformer'
        self.d_model = d_model

        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        self.decoder = nn.Linear(d_model, vocab_size)

        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        nn.init.uniform_(self.embedding.weight, -initrange, initrange)
        nn.init.zeros_(self.decoder.bias)
        nn.init.uniform_(self.decoder.weight, -initrange, initrange)

    def forward(self, src):
        src = self.embedding(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src)
        output = self.decoder(output)
        return output

def decode(input_tokens, inv_vocab):
    return ' '.join([inv_vocab[x] for x in input_tokens])


def save_checkpoint(model, optimizer, episode, folder, filename):
    """
    Save the model and optimizer state to the designated filepath.

    Args:
        model (nn.Module): The model to save.
        optimizer (torch.optim.Optimizer): The optimizer whose state to save.
        episode (int): The current episode number.
        filepath (str): The path where to save the checkpoint.
    """
    filepath = os.path.join(folder, filename)
    # Ensure the directory exists
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    # Save the checkpoint
    torch.save({
        'episode': episode,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, filepath)
    print(f"Checkpoint saved at episode {episode} to {filepath}")

def load_checkpoint(filepath, model, optimizer):
    """
    Load the model and optimizer state from the designated filepath.

    Args:
        filepath (str): The path from where to load the checkpoint.
        model (nn.Module): The model into which to load the state_dict.
        optimizer (torch.optim.Optimizer): The optimizer into which to load the state.

    Returns:
        int: The episode number to resume from.
    """
    if os.path.isfile(filepath):
        checkpoint = torch.load(filepath, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        episode = checkpoint['episode']
        print(f"Checkpoint loaded from {filepath}, resuming from episode {episode}")
        return episode
    else:
        print(f"No checkpoint found at {filepath}, starting from scratch.")
        return 0

In [3]:
def compute_bellman_returns(raw_rewards, gamma):
    bellman_returns = []
    R = 0
    for r in raw_rewards[::-1]:
        R = r + gamma * R
        bellman_returns.insert(0, R)
    return bellman_returns

# Training Loop
def train(verbose=False):
    # Removed torch.autograd.set_detect_anomaly(True)
    vocab_size = len(vocab)
    model = TransformerModel(vocab_size).to(device)
    optimizer = optim.Adam(model.parameters(), lr=2.3e-5)  # Reduced learning rate
    scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=LR_SCHEDULER_GAMMA)
    
    # Optionally, load a checkpoint
    checkpoint_path = os.path.join(OUTPUT_DIR, "ckpt_36000_0.6820_426.26.pth")
    load_checkpoint(checkpoint_path, model, optimizer)

    episode_cnt = 0
    total_reward = 0.0
    num_successes = 0
    success_step_cnts = []
    
    for episode in range(NUM_EPISODES):
        t1 = time.time()
        model.train()  # Set model to training mode
        env = SortingEnv()
        initial_token_id, current_list = env.reset()
        input_tokens = [initial_token_id]
        log_probs = []
        rewards = []
        comparisons = []
        steps = 0
        
        state = 'expect_action'
        done = False
        success = False

        while not done and len(input_tokens) < env.max_steps:
            if verbose:
                print(decode(input_tokens, inv_vocab))
                print(env.get_list())
                print(comparisons)
            # Prepare input tensor
            input_seq = torch.tensor(input_tokens, dtype=torch.long, device=device).unsqueeze(1)  # (seq_len, batch_size)
            # Get model output
            with torch.autocast(device_type=device.type, dtype=torch.bfloat16):
                output = model(input_seq)  # (seq_len, batch_size, vocab_size)
                # Get logits for the last token
                logits = output[-1, 0, :]  # (vocab_size)

                # Check for NaNs in logits
                if torch.isnan(logits).any():
                    print(f"Episode {episode}, NaNs in logits before masking.")
                    break

                # Get valid tokens based on state
                def get_valid_tokens(state):
                    action_tokens = [vocab['Comparison'], vocab['Swap']]
                    index_tokens = [vocab[str(i)] for i in range(env.length)]
                    if state == 'expect_action':
                        return action_tokens
                    elif state == 'expect_index1':
                        return index_tokens[:-1]
                    elif state == 'expect_index2':
                        return [x for x in index_tokens if x > input_tokens[-1]]
                    else:
                        # Handle unexpected states by defaulting to expect_action
                        return action_tokens

                valid_token_ids = get_valid_tokens(state)

                # Ensure valid_token_ids are within the vocab range
                if any(idx >= vocab_size or idx < 0 for idx in valid_token_ids):
                    print(f"Episode {episode}, invalid indices in valid_token_ids: {valid_token_ids}")
                    break

                # Mask invalid tokens
                mask_value = -1e9  # Use a large negative value instead of -inf
                mask = torch.full_like(logits, mask_value).to(device)
                mask[valid_token_ids] = 0
                masked_logits = logits + mask

                # Sample action. Have some chance to randomly pick a valid action.
                eps_threshold = EPS_END + (EPS_START - EPS_END) * np.exp(-1.0 * episode / EPS_DECAY)
                if random.random() < eps_threshold:
                    masked_logits = masked_logits / 4

                # Check for NaNs in masked_logits
                if torch.isnan(masked_logits).any():
                    print(f"Episode {episode}, NaNs in masked_logits after masking.")
                    break

                # Compute probabilities
                probs = F.softmax(masked_logits, dim=0)

                # Check for NaNs in probs
                if torch.isnan(probs).any():
                    print(f"Episode {episode}, NaNs in probs after softmax.")
                    break

                try:
                    m = torch.distributions.Categorical(probs)
                    action_token = m.sample()
                    log_prob = m.log_prob(action_token)
                except ValueError as e:
                    print(f"Episode {episode}, error in sampling action: {e}")
                    break

            log_probs.append(log_prob)
            input_tokens.append(action_token.item())

            action = action_token.item()
            reward = 0.0
            if state == 'expect_action':
                if action == vocab['Comparison']:
                    state = 'expect_index1'
                elif action == vocab['Swap']:
                    if env.indices is None:
                        reward = INVALID_ACTION_REWARD
                        rewards.append(reward)
                        done = True
                        continue
                    action_tokens = [vocab['Swap']]
                    response_token, reward, done, current_list = env.step(action_tokens)
                    steps += 1
                    if done and reward == SUCCESS_REWARD:
                        success = True
                        if episode % 100 == 0:
                            print(decode(input_tokens, inv_vocab))
                    if verbose:
                        print("Reward:", reward)
                    state = 'expect_action'
                else:
                    reward = INVALID_ACTION_REWARD
                    done = True
            elif state == 'expect_index1':
                index1_token = action_token
                state = 'expect_index2'
            elif state == 'expect_index2':
                index2_token = action_token
                action_tokens = [vocab['Comparison'], index1_token.item(), index2_token.item()]
                comparisons.append((int(inv_vocab[index1_token.item()]), 
                                    int(inv_vocab[index2_token.item()])))
                response_token, reward, done, current_list = env.step(action_tokens)
                steps += 1
                if done and reward == SUCCESS_REWARD:
                    success = True
                    if episode % 100 == 0:
                        print(1, decode(input_tokens, inv_vocab))
                else:
                    reward += COMPARISON_ENTROPY_MULTIPLIER * compute_min_delta_entropy(comparisons)
                if verbose:
                    print("Reward:", reward)
                if response_token is not None:
                    input_tokens.append(response_token)
                state = 'expect_action'
            else:
                reward = INVALID_ACTION_REWARD
                done = True

            rewards.append(reward)
        #
        # Post processing of each episode
        success_rewards = [0.0] * len(rewards)
        if success: 
            num_successes += 1
            success_rewards[-1] = SUCCESS_REWARD
            success_step_cnts.append(steps)
        #
        assert len(log_probs) == len(rewards), "log_probs and returns have different sizes!"

        if len(log_probs) == 0:
            continue  # Skip if no actions were taken

        # Compute returns and loss within autocast
        with torch.autocast(device_type=device.type, dtype=torch.bfloat16):
            # Compute returns
            returns1 = compute_bellman_returns(rewards, SHORTTERM_GAMMA)
            returns2 = compute_bellman_returns(success_rewards, LONGTERM_GAMMA)
            returns = torch.tensor(np.array(returns1) + np.array(returns2)).to(device)

            # Check for NaNs in returns
            if torch.isnan(returns).any():
                print(f"Episode {episode}, NaNs in returns.")
                continue

            # Compute loss
            loss = 0
            for log_prob, R in zip(log_probs, returns):
                loss -= log_prob * R

            # Check for NaNs in loss
            if torch.isnan(loss):
                print(f"Episode {episode}, NaN in loss.")
                continue

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping
        optimizer.step()

        episode_cnt += 1
        total_reward += sum(rewards)
        t2 = time.time()
        if episode % 1 == 0:
            print(f"Episode {episode}, loss:{loss.item():.4f}, {'succeed' if success else 'fail'}, steps:{steps}, total reward:{sum(rewards):.4f}, {t2-t1} sec")
            
        # Save checkpoint
        if episode > 0 and episode % EPISODES_SAVE == 0:
            avg_reward = total_reward / episode_cnt
            #assert len(success_step_cnts) == num_successes
            success_rate = num_successes / episode_cnt
            avg_steps = sum(success_step_cnts) / len(success_step_cnts)
            episode_cnt = 0
            total_reward = 0.0
            num_successes = 0
            total_steps = 0
            save_checkpoint(model, optimizer, episode, OUTPUT_DIR, f"ckpt_{episode}_{success_rate:.4f}_{avg_steps:.2f}.pth")
            #
            # Reduce the lr
            scheduler.step()
            # Optionally, log the learning rate
            current_lr = scheduler.get_last_lr()[0]
            print(f"Learning rate = {current_lr:.6f}")

if __name__ == "__main__":
    train(verbose=False)


/home/mcwave/.local/lib/python3.10/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Checkpoint loaded from /home/mcwave/code/autocode/datasets/rl_sort_transformer_easy/list16_transformer4_192_gamma07_step640_v3/ckpt_36000_0.6820_426.26.pth, resuming from episode 36000
Episode 0, loss:-12.1249, fail, steps:205, total reward:14.7000, 2.1134471893310547 sec
Episode 1, loss:-18.4284, fail, steps:205, total reward:16.5000, 1.5575354099273682 sec
Episode 2, loss:-12.8559, fail, steps:201, total reward:11.2000, 1.6232504844665527 sec
Episode 3, loss:-3.7204, fail, steps:205, total reward:17.2000, 1.660752534866333 sec
Episode 4, loss:-37.2764, fail, steps:199, total reward:2.5000, 1.7681288719177246 sec
Episode 5, loss:2.4569, succeed, steps:176, total reward:5.5000, 1.5079686641693115 sec
Episode 6, loss:-13.2953, succeed, steps:175, total reward:-7.5000, 1.566814661026001 sec
Episode 7, loss:7.1159, succeed, steps:199, total reward:9.1000, 1.7064220905303955 sec
Episode 8, loss:-33.6386, fail, steps:192, total reward:-1.7000, 1.6696302890777588 sec
Episode 9, loss:-39.7963

Episode 90, loss:-0.8651, succeed, steps:146, total reward:-1.9000, 1.3780906200408936 sec
Episode 91, loss:-20.0591, fail, steps:205, total reward:14.4000, 1.8467352390289307 sec
Episode 92, loss:12.1040, succeed, steps:127, total reward:7.6000, 1.1219196319580078 sec
Episode 93, loss:4.9097, succeed, steps:107, total reward:2.5000, 0.8583855628967285 sec
Episode 94, loss:20.5262, succeed, steps:195, total reward:31.4000, 1.4312350749969482 sec
Episode 95, loss:-9.6437, fail, steps:207, total reward:19.5000, 1.7678391933441162 sec
Episode 96, loss:-32.9448, fail, steps:198, total reward:4.5000, 1.6864008903503418 sec
Episode 97, loss:-21.0945, fail, steps:202, total reward:10.7000, 1.7238752841949463 sec
Episode 98, loss:1.7919, succeed, steps:170, total reward:9.6000, 1.4617316722869873 sec
Episode 99, loss:-18.0128, fail, steps:205, total reward:16.0000, 1.8613512516021729 sec
Episode 100, loss:-18.5110, fail, steps:205, total reward:16.0000, 1.6974968910217285 sec
Episode 101, loss

Episode 181, loss:2.5503, succeed, steps:192, total reward:6.9000, 1.7308392524719238 sec
Episode 182, loss:-22.7044, fail, steps:200, total reward:9.0000, 1.7265114784240723 sec
Episode 183, loss:-3.3740, succeed, steps:149, total reward:0.3000, 1.3452038764953613 sec
Episode 184, loss:-30.1162, fail, steps:195, total reward:-2.5000, 1.825821876525879 sec
Episode 185, loss:25.6338, succeed, steps:190, total reward:29.3000, 1.5035817623138428 sec
Episode 186, loss:19.5945, succeed, steps:193, total reward:23.5000, 1.6486856937408447 sec
Episode 187, loss:11.9805, succeed, steps:192, total reward:19.4000, 1.6038494110107422 sec
Episode 188, loss:16.2482, succeed, steps:203, total reward:20.9000, 1.704829216003418 sec
Episode 189, loss:10.3301, succeed, steps:186, total reward:13.6000, 1.5543601512908936 sec
Episode 190, loss:10.0821, succeed, steps:187, total reward:16.1000, 1.5101289749145508 sec
Episode 191, loss:-14.6805, fail, steps:203, total reward:14.5000, 1.8178822994232178 sec


Episode 243, loss:-22.6979, fail, steps:197, total reward:1.7000, 1.9271876811981201 sec
Episode 244, loss:-38.8428, fail, steps:196, total reward:0.7000, 1.7080776691436768 sec
Episode 245, loss:13.6587, succeed, steps:145, total reward:13.2000, 1.2164998054504395 sec
Episode 246, loss:10.4879, succeed, steps:192, total reward:11.9000, 1.6333484649658203 sec
Episode 247, loss:1.2403, succeed, steps:168, total reward:14.8000, 1.2651076316833496 sec
Episode 248, loss:-13.1353, fail, steps:204, total reward:15.5000, 1.7758331298828125 sec
Episode 249, loss:-2.4513, succeed, steps:187, total reward:12.3000, 1.5496606826782227 sec
Episode 250, loss:3.2827, succeed, steps:199, total reward:13.9000, 1.7679364681243896 sec
Episode 251, loss:-32.3780, fail, steps:200, total reward:9.3000, 1.7252285480499268 sec
Episode 252, loss:6.2229, succeed, steps:157, total reward:18.7000, 1.2982289791107178 sec
Episode 253, loss:1.1644, succeed, steps:167, total reward:7.9000, 1.3394825458526611 sec
Epis

...

Episode 9923, loss:-1.5596, succeed, steps:180, total reward:11.1000, 2.4831337928771973 sec
Episode 9924, loss:1.4639, succeed, steps:127, total reward:10.0000, 1.6989977359771729 sec
Episode 9925, loss:-9.1559, fail, steps:197, total reward:5.9000, 3.056001901626587 sec
Episode 9926, loss:2.0680, succeed, steps:196, total reward:23.3000, 2.7372264862060547 sec
Episode 9927, loss:1.4816, succeed, steps:168, total reward:22.0000, 2.2492411136627197 sec
Episode 9928, loss:4.2923, succeed, steps:172, total reward:5.9000, 2.2970781326293945 sec
Episode 9929, loss:5.6748, succeed, steps:210, total reward:35.4000, 2.786735773086548 sec
Episode 9930, loss:6.2102, succeed, steps:161, total reward:24.0000, 2.2511205673217773 sec
Episode 9931, loss:8.5548, succeed, steps:184, total reward:26.0000, 2.408782720565796 sec
Episode 9932, loss:-5.7194, fail, steps:208, total reward:22.1000, 3.0733511447906494 sec
Episode 9933, loss:-1.5529, fail, steps:202, total reward:13.5000, 2.8588290214538574 se

Episode 10000, loss:-2.3598, succeed, steps:134, total reward:8.7000, 1.6675148010253906 sec
Checkpoint saved at episode 10000 to /home/mcwave/code/autocode/datasets/rl_sort_transformer_easy/list16_transformer4_192_gamma07_step640_v3/ckpt_10000_0.7810_169.08.pth
Learning rate = 0.000016
Episode 10001, loss:5.9176, succeed, steps:176, total reward:22.6000, 2.071591854095459 sec
Episode 10002, loss:6.6795, succeed, steps:191, total reward:25.8000, 2.4786181449890137 sec
Episode 10003, loss:2.9649, succeed, steps:159, total reward:25.6000, 1.951111078262329 sec
Episode 10004, loss:2.8647, succeed, steps:107, total reward:15.5000, 1.2939081192016602 sec
Episode 10005, loss:5.4707, succeed, steps:187, total reward:44.8000, 2.1046946048736572 sec
Episode 10006, loss:5.8218, succeed, steps:194, total reward:31.4000, 2.36258602142334 sec
Episode 10007, loss:5.3133, succeed, steps:166, total reward:36.2000, 1.9388511180877686 sec
Episode 10008, loss:3.0077, succeed, steps:184, total reward:21.4

Episode 10087, loss:-0.9051, succeed, steps:193, total reward:24.7000, 2.499650001525879 sec
Episode 10088, loss:2.6782, succeed, steps:135, total reward:11.0000, 1.695303201675415 sec
Episode 10089, loss:-6.9683, fail, steps:207, total reward:16.2000, 3.003647565841675 sec
Episode 10090, loss:0.1925, fail, steps:216, total reward:32.7000, 2.9716193675994873 sec
Episode 10091, loss:1.5945, succeed, steps:207, total reward:23.6000, 2.8985695838928223 sec
Episode 10092, loss:2.9232, succeed, steps:206, total reward:40.4000, 2.7728962898254395 sec
Episode 10093, loss:1.4917, succeed, steps:152, total reward:31.0000, 1.8260526657104492 sec
Episode 10094, loss:-0.4160, fail, steps:212, total reward:25.9000, 2.9858334064483643 sec
Episode 10095, loss:1.2292, succeed, steps:133, total reward:25.9000, 1.6762306690216064 sec
Episode 10096, loss:3.6783, succeed, steps:171, total reward:16.9000, 2.3999881744384766 sec
Episode 10097, loss:3.1040, succeed, steps:181, total reward:36.2000, 2.1679055

Episode 10154, loss:-1.1460, succeed, steps:215, total reward:42.7000, 2.062361717224121 sec



KeyboardInterrupt

